In [2]:
# preprocess_frames.py
import os
import cv2
import numpy as np

# Cấu hình thư mục
data_dir = './'  # Thư mục gốc chứa RAVDESS
output_dir = './processed_frames'  # Thư mục lưu ảnh đã trích

os.makedirs(output_dir, exist_ok=True)

# Hàm trích frame
def extract_frame(video_path, frame_size=(64, 64)):
    cap = cv2.VideoCapture(video_path)
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, 1.1, 5)
        
        if len(faces) > 0:
            (x, y, w, h) = faces[0]
            face = frame[y:y+h, x:x+w]
            face = cv2.resize(face, frame_size)
            cap.release()
            return face
    cap.release()
    return np.zeros((frame_size[0], frame_size[1], 3), dtype=np.uint8)

# Lặp và trích frame
emotions_map = {
    '01': 'neutral', '02': 'calm', '03': 'happy', '04': 'sad',
    '05': 'angry', '06': 'fear', '07': 'disgust', '08': 'surprise'
}

image_paths, labels = [], []

for actor_folder in os.listdir(data_dir):
    actor_path = os.path.join(data_dir, actor_folder)
    if os.path.isdir(actor_path) and (actor_folder.startswith('Video_Speech_Actor_') or actor_folder.startswith('Video_Song_Actor_')):
        for subfolder in os.listdir(actor_path):
            subfolder_path = os.path.join(actor_path, subfolder)
            if os.path.isdir(subfolder_path) and subfolder.startswith('Actor_'):
                for file in os.listdir(subfolder_path):
                    if file.endswith('.mp4'):
                        emotion_code = file.split('-')[2]
                        emotion = emotions_map.get(emotion_code)
                        if emotion:
                            video_path = os.path.join(subfolder_path, file)
                            frame = extract_frame(video_path)

                            # Lưu ảnh
                            output_path = os.path.join(output_dir, file.replace('.mp4', '.jpg'))
                            cv2.imwrite(output_path, cv2.cvtColor(frame, cv2.COLOR_RGB2BGR))

                            image_paths.append(output_path)
                            labels.append(emotion)

# Lưu đường dẫn và nhãn
import pickle
with open("image_paths_labels.pkl", "wb") as f:
    pickle.dump((image_paths, labels), f)


In [4]:
# train_from_images.py
import os
import pickle
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import torchvision.transforms as transforms
from PIL import Image

# Load danh sách ảnh và nhãn từ file
with open("image_paths_labels.pkl", "rb") as f:
    image_paths, labels = pickle.load(f)

# Encode nhãn
le = LabelEncoder()
labels_encoded = torch.tensor(le.fit_transform(labels), dtype=torch.long)

# Tạo transform
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3)
])

# Dataset dùng ảnh đã lưu
class PreprocessedDataset(Dataset):
    def __init__(self, image_paths, labels, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img = cv2.imread(self.image_paths[idx])
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        if self.transform:
            img = self.transform(img)
        return img, self.labels[idx]

# Chia train/test
train_paths, test_paths, train_labels, test_labels = train_test_split(
    image_paths, labels_encoded, test_size=0.3, random_state=42, stratify=labels_encoded
)

train_dataset = PreprocessedDataset(train_paths, train_labels, transform)
test_dataset = PreprocessedDataset(test_paths, test_labels, transform)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)  # ⚡ Thêm num_workers
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=4)

# Mô hình CNN
class FacialEmotionCNN(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.model = nn.Sequential(
            nn.Conv2d(3, 16, 3, padding=1), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(16, 32, 3, padding=1), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(32, 64, 3, padding=1), nn.ReLU(), nn.MaxPool2d(2),
            nn.Flatten(),
            nn.Linear(64*8*8, 64), nn.ReLU(), nn.Dropout(0.3),
            nn.Linear(64, num_classes)
        )

    def forward(self, x):
        return self.model(x)

# Training
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = FacialEmotionCNN(num_classes=len(le.classes_)).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

def train_model(model, loader, criterion, optimizer, epochs=300):
    model.train()
    for epoch in range(epochs):
        total, correct, running_loss = 0, 0, 0
        for inputs, labels in loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss:.4f}, Acc: {100*correct/total:.2f}%")

    torch.save(model.state_dict(), "facial_emotion_cnn.pth")
    print("✅ Model saved to facial_emotion_cnn.pth")

def evaluate_model(model, loader):
    model.eval()
    total, correct = 0, 0
    with torch.no_grad():
        for inputs, labels in loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    print(f"🎯 Test Accuracy: {100*correct/total:.2f}%")

# Train và evaluate
train_model(model, train_loader, criterion, optimizer)
evaluate_model(model, test_loader)


Epoch 1/300, Loss: 221.0835, Acc: 16.23%
Epoch 2/300, Loss: 215.4765, Acc: 22.00%
Epoch 3/300, Loss: 189.8262, Acc: 32.78%
Epoch 4/300, Loss: 168.1934, Acc: 39.04%
Epoch 5/300, Loss: 148.0661, Acc: 47.61%
Epoch 6/300, Loss: 138.2027, Acc: 51.49%
Epoch 7/300, Loss: 127.3867, Acc: 55.65%
Epoch 8/300, Loss: 116.4636, Acc: 59.24%
Epoch 9/300, Loss: 110.3359, Acc: 61.36%
Epoch 10/300, Loss: 105.8956, Acc: 62.09%
Epoch 11/300, Loss: 96.7036, Acc: 65.33%
Epoch 12/300, Loss: 93.2126, Acc: 66.78%
Epoch 13/300, Loss: 85.2211, Acc: 68.68%
Epoch 14/300, Loss: 80.3859, Acc: 70.51%
Epoch 15/300, Loss: 81.6038, Acc: 70.80%
Epoch 16/300, Loss: 78.5739, Acc: 71.47%
Epoch 17/300, Loss: 73.9165, Acc: 72.93%
Epoch 18/300, Loss: 69.8008, Acc: 73.98%
Epoch 19/300, Loss: 68.7824, Acc: 73.95%
Epoch 20/300, Loss: 66.3907, Acc: 75.41%
Epoch 21/300, Loss: 63.4566, Acc: 76.34%
Epoch 22/300, Loss: 58.8468, Acc: 77.83%
Epoch 23/300, Loss: 61.3428, Acc: 76.72%
Epoch 24/300, Loss: 59.1806, Acc: 78.09%
Epoch 25/300, L

In [1]:
# 1. IMPORT LIBRARIES
import os
import cv2
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import torchvision.transforms as transforms

# 2. EXTRACT FRAME FUNCTION (resize nhỏ hơn)
def extract_frame(video_path, frame_size=(64, 64)):
    cap = cv2.VideoCapture(video_path)
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, 1.1, 5)
        if len(faces) > 0:
            (x, y, w, h) = faces[0]
            face = frame[y:y+h, x:x+w]
            face = cv2.resize(face, frame_size)
            face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
            cap.release()
            return face
    cap.release()
    return np.zeros((frame_size[0], frame_size[1], 3), dtype=np.uint8)

# 3. CUSTOM DATASET
class RAVDESSDataset(Dataset):
    def __init__(self, video_paths, labels, transform=None):
        self.video_paths = video_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.video_paths)

    def __getitem__(self, idx):
        frame = extract_frame(self.video_paths[idx])
        if self.transform:
            frame = self.transform(frame)
        return frame, self.labels[idx]

# 4. LOAD DATASET (RAVDESS)
data_dir = './'  # Thư mục chứa dataset
emotions_map = {
    '01': 'neutral', '02': 'calm', '03': 'happy', '04': 'sad',
    '05': 'angry', '06': 'fear', '07': 'disgust', '08': 'surprise'
}

video_paths, labels = [], []
for actor_folder in os.listdir(data_dir):
    actor_path = os.path.join(data_dir, actor_folder)
    if os.path.isdir(actor_path) and (actor_folder.startswith('Video_Speech_Actor_') or actor_folder.startswith('Video_Song_Actor_')):
        for subfolder in os.listdir(actor_path):
            subfolder_path = os.path.join(actor_path, subfolder)
            if os.path.isdir(subfolder_path) and subfolder.startswith('Actor_'):
                for file in os.listdir(subfolder_path):
                    if file.endswith('.mp4'):
                        emotion_code = file.split('-')[2]
                        emotion = emotions_map.get(emotion_code)
                        if emotion:
                            video_paths.append(os.path.join(subfolder_path, file))
                            labels.append(emotion)

# 5. ENCODE LABELS + GIỚI HẠN DỮ LIỆU
le = LabelEncoder()
labels_encoded = torch.tensor(le.fit_transform(labels), dtype=torch.long)
video_paths = video_paths
labels_encoded = labels_encoded

# 6. TRANSFORMS
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3)
])

# 7. SPLIT DATASET
train_paths, test_paths, train_labels, test_labels = train_test_split(
    video_paths, labels_encoded, test_size=0.3, random_state=42, stratify=labels_encoded
)
train_dataset = RAVDESSDataset(train_paths, train_labels, transform)
test_dataset = RAVDESSDataset(test_paths, test_labels, transform)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# 8. CNN MODEL (nhẹ hơn)
class FacialEmotionCNN(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.model = nn.Sequential(
            nn.Conv2d(3, 16, 3, padding=1), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(16, 32, 3, padding=1), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(32, 64, 3, padding=1), nn.ReLU(), nn.MaxPool2d(2),
            nn.Flatten(),
            nn.Linear(64*8*8, 64), nn.ReLU(), nn.Dropout(0.3),
            nn.Linear(64, num_classes)
        )

    def forward(self, x):
        return self.model(x)

# 9. TRAINING
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = FacialEmotionCNN(num_classes=len(le.classes_)).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

def train_model(model, loader, criterion, optimizer, epochs=100):
    model.train()
    for epoch in range(epochs):
        total, correct, running_loss = 0, 0, 0
        for inputs, labels in loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss:.4f}, Acc: {100*correct/total:.2f}%")
        # Save model
    model_path = "facial_emotion_cnn.pth"
    torch.save(model.state_dict(), model_path)
    print(f"Model saved to {model_path}")

train_model(model, train_loader, criterion, optimizer)

# 10. EVALUATE
def evaluate_model(model, loader):
    model.eval()
    total, correct = 0, 0
    with torch.no_grad():
        for inputs, labels in loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    print(f"Test Accuracy: {100*correct/total:.2f}%")

evaluate_model(model, test_loader)


KeyboardInterrupt: 